EDA y limpieza de datos

Carga de librerías

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Carga de datos a limpiar

In [4]:
df_experiment=pd.read_csv ("Proyectos semanales/Vanguard/df_final_experiment_clients.txt") 
df_data=pd.read_csv ("Proyectos semanales/Vanguard/df_data.txt") 

Unimos los dataframes de data, son los que nos aportan información del funnel

In [5]:
#Unimos los dos df con id de cliente en común
df_beh = pd.merge(df_data, df_experiment, on='client_id', how='inner')

In [11]:
df_beh

,client_id,visitor_id,visit_id,process_step,date_time,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test
...,...,...,...,...,...,...
449826,9895983,473024645_56027518531,498981662_93503779869_272484,step_3,2017-06-15 19:52:09,UN
449827,9895983,473024645_56027518531,498981662_93503779869_272484,step_2,2017-06-15 19:50:37,UN
449828,9895983,473024645_56027518531,498981662_93503779869_272484,step_1,2017-06-15 19:50:05,UN
449829,9895983,473024645_56027518531,498981662_93503779869_272484,start,2017-06-15 19:50:00,UN


In [7]:
df_beh['Variation'] = df_beh['Variation'].fillna('UN')

In [14]:
df_beh['date_time'] = df_beh['date_time'].fillna('0')

In [8]:
#df_beh.to_csv('Proyectos semanales/Vanguard/df_beh.txt', index=False)

Análisis del comportamiento de los usuarios

1. Tasa de Finalización

In [9]:
final_step_users = df_beh[df_beh['process_step'] == 'confirm']['client_id'].nunique()
total_users = df_beh['client_id'].nunique()

completion_rate = final_step_users / total_users
print(f"Tasa de finalización: {completion_rate:.2%}")


Tasa de finalización: 67.70%


2. Tiempo Dedicado a Cada Paso

In [ ]:
# Revisar que la columna de tiempo esta en formato datetime
df_beh['date_time'] = pd.to_datetime(df_beh['date_time'])

# Ordenar por cliente y fecha para calcular la diferencia de tiempo entre pasos
df_beh.sort_values(by=['client_id', 'date_time'], inplace=True)

# Calcular la diferencia de tiempo
df_beh['time_spent'] = df_beh.groupby('client_id')['date_time'].diff()

# Calcular el tiempo medio dedicado a cada paso
average_time_per_step = df_beh.groupby('process_step')['time_spent'].mean()

print("Tiempo promedio dedicado a cada paso:")
print(average_time_per_step)


Tiempo promedio dedicado a cada paso:
process_step
confirm   1 days 00:24:15.330060473
start     3 days 15:30:50.642098449
step_1    0 days 00:29:20.193023912
step_2    0 days 00:01:14.071613535
step_3    0 days 00:11:29.029775618
Name: time_spent, dtype: timedelta64[ns]


3. Tasas de Error

In [12]:
# Definir un orden numérico a cada paso
step_order = {'start': 1, 'step_1': 2, 'step_2': 3, 'step_3': 4, 'confirmación': 5} 

df_beh['step_number'] = df_beh['process_step'].map(step_order)

# Marcar si cada paso es un error (retorno a un paso anterior)
df_beh['is_error'] = df_beh.groupby('client_id')['step_number'].diff() < 0

# Calcular la tasa de error por paso
error_rates = df_beh[df_beh['is_error']].groupby('process_step').size() / df_beh.groupby('process_step').size()

print("Tasa de error por paso:")
print(error_rates)


Tasa de error por paso:
process_step
confirm         NaN
start      0.580096
step_1     0.730208
step_2     0.799977
step_3          NaN
dtype: float64


In [ ]:
#df_users.to_csv('Proyectos semanales/Vanguard/df_users.txt', index=False)